In [1]:
import nltk 
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
processed_df = pd.read_pickle("../data/processed/processed_v2.pkl")

In [4]:
processed_inbound_extra = processed_df["clean_inbound_text"]

In [7]:
def train_doc2vec(string_data, max_epochs, vec_size, alpha):
     
    # Tagging each of the documents with a unique ID
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(string_data)]
    
    # Instantiating my model 
    model = Doc2Vec(vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm=1) # dm=1 means 'distributed memory' (PV-DM)
    
    # Building the vocabulary table
    model.build_vocab(tagged_data)
    
    for epoch in range(max_epochs): # Run for max_epochs
        print('iteration {0}'.format(epoch))    
        model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs) # This statement trains the model on the current epoch
        # Decreasing the learning rate
        model.alpha -= 0.0002
        # Fixing the learning rate, no decay
        model.min_alpha = model.alpha
        
    # Saving model
    model.save("../modelsmodels/d2v.model")
    print("Model Saved")
    
# Training
train_doc2vec(processed_inbound_extra, max_epochs=100, vec_size=20, alpha=0.025)

iteration 0


KeyboardInterrupt: 

In [ ]:
model = Doc2Vec()

In [1]:
# Creating a synthetic dataset - generating N Tweets resembling a mock tweet
# This will subsequently be merged with the existing inbound data for inclusion in the doc2vec training process

# Version 1 - will be improved in future iterations
ideal = {
            "order track": "@AmazonHelo Hi, could you provide an update on the order? Its been days since the product has moved from its last location ", # change intent to "order tracking"???
            "product inquiry": "@AmazonHelp Looking for more info on the product. Can you share details or direct me to a reliable source?", # product inquire??? 
            "return refund": "@AmazonHelp How can I start a return process? The item I received doesn't match the description.",
            "account management": "@AmazonHelp Hi, I am having trouble logging into my account. Can you help me reset my password?", 
            "promotion discount": "@AmazonHelp Are there any ongoing promotions or deals in the ongoing festive season? Looking to buy a few items.",
            "shipping": "@AmazonHelp Hi, My address has changed. Can you help me update the shipping address for my order?",
            "technical support": "@AmazonHelp Encountering errors during checkout. Can you help me troubleshoot the issue?",
            "payment issue": "@AmazonHelp My payment method isn't going through. Any suggestions on how to resolve this?",
            "general query": "@AmazonHelp Hi, I have a general question regarding the product. Can you help me with this?"
        }

In [ ]:
def add_extra(current_tokenized_data, extra_tweets):
    '''Adding extra tweets to the current tokenized data'''
    
    # Convert the extra tweets into a pandas Series
    extra_tweets = pd.Series(extra_tweets)

    # Convert the current tokenized data into a single string
    print('Converting to string...')
    string_processed_data = current_tokenized_data.progress_apply(" ".join)

    # Concatenate the extra tweets to the current data
    string_processed_data = pd.concat([string_processed_data, extra_tweets], axis = 0)

    # Tokenize the combined data
    tknzr = TweetTokenizer(strip_handles = True, reduce_len = True)
    print('Tokenizing...')
    tokenized_data = string_processed_data.progress_apply(tknzr.tokenize)

    return tokenized_data

# Add the extra tweets to the current data
processed_inbound_extra = add_extra(processed['Processed Inbound'], list(ideal.values()))

# Save the updated data to a pickle file
processed_inbound_extra.to_pickle('objects/processed_inbound_extra.pkl')

processed_inbound_extra

Here's is the catch. Have intent buckets inplace already in place. You would need to supply this to the below function to get your top N tweets corresponding to the current tweet


In [ ]:
# Dictionary mapping the intent to the row index 
intent_itags = {

}

def generate_intent(nsim, idx_tag): 
    '''Function that maps an index tag to an intent and returns nsim number of similar tweets'''
    sim_docs = model.docvecs.most_similar(idx_tag, topn = nsim)
    
    # Getting just the indexes 
    indexes = [int(i[0]) for i in sim_docs]
    
    # Actually seeing the top 1000 tweets similar to 0th tweet which seems to be about updates 
    # print(processed_inbound_extra[indexes])
    return indexes
    
# Create a dictionary mapping the intent to the row index of tweets
index_intents = {}
for intent, tag in intent_itags.items():
    print('Intent: ', intent)
    index_indents[intent] = generate_intent(1000, tag)
    print('\n')

In [ ]:
# Now map the index to each row of the preprocessed inbound data
preprocessed_inbound["intent"] = processed_inbounnd.index.map(index_intents)

In [ ]:
# Intent classification with Keras 
